In [1]:
import pandas as pd
import json

with open('/content/DataEngineeringQ2.json', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data)
print(df.head())


        _id appointmentId  phoneNumber patientDetails._id  \
0  T6hf3rb5     40d2-9c9f  96686896670          T6hb630b3   
1  T65g3rb5      40dbtc9f   9496368916          T6h33b300   
2  T7g6Srb5      g3Wt5c9f   7787204833           TjhB4373   
3  94bg8W8d      fb6-a535   9376756879            6df4R5b   
4  T6h8H56c       67h7KL9   5267384241           lK9hy06a   

  patientDetails.firstName patientDetails.lastName patientDetails.emailId  \
0                      Css                                                  
1                   Lokesh                                                  
2                    Shila                     Das                          
3                  Bhavika             Ben Panchal                          
4               Raghu Viju                                                  

  patientDetails.gender patientDetails.alternateContact  \
0                                                         
1                     M                             

In [2]:
# Extract medicines information and count active medicines
df['active_medicines_count'] = df['consultationData.medicines'].apply(
    lambda medicines: sum(1 for med in medicines if med['isActive'])
)

# Display the transformed data
print(df[['patientDetails.firstName', 'patientDetails.lastName', 'active_medicines_count']])


    patientDetails.firstName patientDetails.lastName  active_medicines_count
0                        Css                                               1
1                     Lokesh                                               1
2                      Shila                     Das                       2
3                    Bhavika             Ben Panchal                       1
4                 Raghu Viju                                               2
5               Dinesh Kumar                                               2
6                      Lalit                Sankhwal                       2
7                       Ravi                                               1
8                     Dinesh                                               1
9                     Sanjay                                               0
10                    Akshay                  Akshay                       0
11                   Nirmala                                               2

In [3]:
# Check for missing values in critical columns
missing_values = df[['patientDetails.firstName', 'consultationData.medicines']].isnull().sum()
print(missing_values)

# Validate phone numbers to ensure they contain only digits and have a length of 10-15
df['phoneNumber_valid'] = df['phoneNumber'].apply(lambda x: x.isdigit() and 10 <= len(x) <= 15)
print(df[['phoneNumber', 'phoneNumber_valid']])


patientDetails.firstName      0
consultationData.medicines    0
dtype: int64
       phoneNumber  phoneNumber_valid
0      96686896670               True
1       9496368916               True
2       7787204833               True
3       9376756879               True
4       5267384241               True
5    +919826374025              False
6       7587265252               True
7       5586958767               True
8       9987602525               True
9       9234354366               True
10      4863281056               True
11      8687986800               True
12                              False
13      7152684236               True
14                              False
15      9184723620               True
16                              False
17      5847346075               True
18      9064537237               True
19                              False
20       992727892              False
21      6888324121               True
22      8248594521               True
23   +91793

In [4]:
# Calculate the average number of medicines prescribed per patient
df['total_medicines_count'] = df['consultationData.medicines'].apply(len)
average_medicines_per_patient = df['total_medicines_count'].mean()
print(f'Average number of medicines prescribed per patient: {average_medicines_per_patient:.2f}')


Average number of medicines prescribed per patient: 2.13


In [5]:


# Calculate the percentage of missing values for specific columns
columns_to_check = ['patientDetails.firstName', 'patientDetails.lastName', 'patientDetails.birthDate']
missing_percentages = df[columns_to_check].isnull().mean() * 100

print(missing_percentages)


patientDetails.firstName     0.000000
patientDetails.lastName      0.000000
patientDetails.birthDate    32.258065
dtype: float64


In [7]:
# Replace empty strings with NaN
df['patientDetails.gender'].replace('', pd.NA, inplace=True)

# Calculate the mode of the gender column again
gender_mode = df['patientDetails.gender'].mode()[0]

# Fill missing values in the gender column with the mode
df['patientDetails.gender'].fillna(gender_mode, inplace=True)

# Calculate the percentage of female gender, considering 'F' as Female
female_percentage = (df['patientDetails.gender'] == 'F').mean() * 100

print(female_percentage)



32.25806451612903


In [8]:
import pandas as np

# Convert birthDate to datetime
df['patientDetails.birthDate'] = pd.to_datetime(df['patientDetails.birthDate'], errors='coerce')

# Calculate age
current_year = pd.to_datetime('now').year
df['age'] = current_year - df['patientDetails.birthDate'].dt.year

# Define age groups
def categorize_age(age):
    if age < 0:
        return np.nan
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'

# Apply the categorize_age function
df['ageGroup'] = df['age'].apply(categorize_age)

# Count the number of adults
adult_count = df['ageGroup'].value_counts().get('Adult', 0)

print(adult_count)


21


In [9]:

# Calculate the total number of medicines prescribed for each patient
df['total_medicines_count'] = df['consultationData.medicines'].apply(len)

# Calculate the average number of medicines prescribed
average_medicines_per_patient = df['total_medicines_count'].mean()

print(f'Average number of medicines prescribed per patient: {average_medicines_per_patient:.2f}')


Average number of medicines prescribed per patient: 2.13


In [11]:

from collections import Counter

# Normalize the JSON data into a flat table
df = pd.json_normalize(data)

# Extract all medicine names into a list
all_medicine_names = []
for medicines in df['consultationData.medicines']:
    all_medicine_names.extend([medicine['medicineName'] for medicine in medicines])

# Count the occurrences of each medicine name
medicine_counter = Counter(all_medicine_names)

# Find the 3rd most common medicine name
third_most_common_medicine = medicine_counter.most_common(3)[2][0]

print(third_most_common_medicine)


C


In [12]:


# Extract all medicine statuses into a list
active_medicines_count = 0
inactive_medicines_count = 0

for medicines in df['consultationData.medicines']:
    for medicine in medicines:
        if medicine['isActive']:
            active_medicines_count += 1
        else:
            inactive_medicines_count += 1

# Calculate the total number of medicines
total_medicines_count = active_medicines_count + inactive_medicines_count

# Calculate the percentage distribution
active_percentage = (active_medicines_count / total_medicines_count) * 100
inactive_percentage = (inactive_medicines_count / total_medicines_count) * 100

(active_percentage, inactive_percentage)


(69.6969696969697, 30.303030303030305)

In [13]:

# Calculate the percentage of missing values for specific columns
columns_to_check = ['patientDetails.firstName', 'patientDetails.lastName', 'patientDetails.birthDate']
missing_percentages = df[columns_to_check].isnull().mean() * 100

print(missing_percentages)


patientDetails.firstName     0.000000
patientDetails.lastName      0.000000
patientDetails.birthDate    32.258065
dtype: float64
